# Sequential Convergence Study Tutorial

This tutorial demonstrates how to run the Sequential Convergence Study experiment, which investigates whether agents starting from uniform initial probability distributions exhibit sequential convergence to degenerate distributions.

## Concept
The sequential convergence study examines whether agents converge to specialised resource preferences one after another rather than simultaneously. This pattern emerges from the partial observability of the system, where early convergers influence the cost landscape for subsequent agents through environmental feedback.

This experiment tests the hypothesis that agents exhibit predictable sequential convergence patterns under uniform initial conditions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime
import sys
import os

# Add the parent directory to the path to import the experiment modules
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from experiments.sequential_convergence_study import SequentialConvergenceStudy, run_sequential_convergence_study
from utils.config import Config
from evaluation.metrics import calculate_entropy, calculate_convergence_speed
from visualisation.plots import plot_convergence_comparison

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Concept Explanation

### What is Sequential Convergence?
Sequential convergence refers to the phenomenon where agents achieve specialisation (degenerate probability distributions) at different time points rather than simultaneously. This creates a temporal hierarchy of convergence events.

### 1. Why Study Sequential Convergence?
- Understanding coordination dynamics under partial observability
- Identifying emergent temporal patterns in multi-agent systems
- Validating theoretical predictions about convergence behaviour
- Exploring the role of environmental feedback in coordination

**Key characteristics:** 
 - Agents converge one after another 
 - Early convergers influence later ones 
  - Environmental feedback drives coordination 
 - Partial observability enables temporal separation

### 2. THEORETICAL FOUNDATION:

Sequential convergence emerges from the partial observability structure of the system. Agents cannot observe each other's probability distributions directly, but they can infer optimal behaviour through environmental feedback. Early convergers create cost gradients that influence subsequent agent decisions, leading to predictable temporal coordination patterns. 
    
### 3. KEY RESEARCH QUESTIONS:

- Do agents converge sequentially rather than simultaneously? 
- What determines the order of convergence? 
- How does environmental feedback influence convergence timing? 
- Are sequential patterns consistent across replications? 
- What is the relationship between convergence order and performance?
    
### 4. EXPECTED OUTCOMES:

- Agents should converge at distinct time points 
- Sequential patterns should be consistent across replications 
- Early convergers should influence later agent behaviour 
- All agents should achieve degenerate distributions 
- Convergence timing should correlate with resource preferences


## 2. Parameter Configuration

In this section, we'll configure the experiment parameters. You can modify these values to explore different scenarios.

In [ ]:
# System parameters
num_agents = 10
num_resources = 3  # Using 3 resources for barycentric analysis
num_iterations = 2000
num_replications = 100

# Learning parameters
weight = 0.3

# Resource capacity configuration (balanced)
relative_capacity = [0.5, 0.5, 0.5]

# Convergence parameters
convergence_threshold_entropy = 0.1
convergence_threshold_max_prob = 0.9

# Output configuration
output_dir = "results/sequential_convergence_tutorial"

# Display configuration
print("=" * 80)
print("EXPERIMENT CONFIGURATION")
print("=" * 80)
print(f"Number of agents: {num_agents}")
print(f"Number of resources: {num_resources}")
print(f"Number of iterations: {num_iterations}")
print(f"Number of replications: {num_replications}")
print(f"Learning rate (weight): {weight}")
print(f"Relative capacity: {[f'{c:.3f}' for c in relative_capacity]}")
print(f"Convergence entropy threshold: {convergence_threshold_entropy}")
print(f"Convergence max probability threshold: {convergence_threshold_max_prob}")
print(f"Output directory: {output_dir}")

## 3. Running the Experiment

Now we'll create and run the sequential convergence experiment.

In [ ]:
def run_sequential_convergence_experiment(params):
    """
    Run the sequential convergence study experiment with the given parameters.
    """
    print("=" * 80)
    print("RUNNING SEQUENTIAL CONVERGENCE STUDY EXPERIMENT")
    print("=" * 80)
    
    # Create output directory
    output_path = Path(params['output_dir'])
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Create and run the experiment
    print(f"Creating SequentialConvergenceStudy...")
    study = SequentialConvergenceStudy(
        results_dir=params['output_dir'],
        experiment_name="sequential_convergence_tutorial"
    )
    
    # Override base config with user parameters
    study.base_config.num_agents = params['num_agents']
    study.base_config.num_resources = params['num_resources']
    study.base_config.num_iterations = params['num_iterations']
    study.base_config.weight = params['weight']
    study.base_config.relative_capacity = params['relative_capacity']
    
    print(f"Running experiment with {params['num_replications']} replications...")
    print("This may take several minutes depending on the number of replications...")
    
    # Run experiment
    full_results = study.run_experiment(num_episodes=params['num_replications'])
    
    # Convert results to expected format
    study.results = []
    for config_result in full_results['results']:
        config_params = config_result['config_params']
        for episode_result in config_result['episode_results']:
            study.results.append({
                'config_params': config_params,
                'simulation_results': episode_result,
                'replication_id': episode_result['episode']
            })
    
    
    return study

# Create parameter dictionary
params = {
    'num_agents': num_agents,
    'num_resources': num_resources,
    'num_iterations': num_iterations,
    'num_replications': num_replications,
    'weight': weight,
    'relative_capacity': relative_capacity,
    'convergence_threshold_entropy': convergence_threshold_entropy,
    'convergence_threshold_max_prob': convergence_threshold_max_prob,
    'output_dir': output_dir
}

# Run the experiment
study = run_sequential_convergence_experiment(params)

## 4. Generating Analysis and Plots

Now we'll generate comprehensive analysis and create visualisations.

In [ ]:
def generate_analysis_and_plots(study, params):
    """
    Generate comprehensive analysis and create all figures.
    """
    print("=" * 80)
    print("GENERATING ANALYSIS AND PLOTS")
    print("=" * 80)
    
    # Generate detailed analysis
    print("Generating detailed statistical analysis...")
    analysis_results = study.perform_comprehensive_analysis()
    
    # Create comprehensive plots
    print("Creating comprehensive visualisations...")
    plots_dir = f"{params['output_dir']}/plots"
    plot_files = study.create_comprehensive_plots(plots_dir)
    
    print(f"Generated {len(plot_files)} plot files:")
    print(f"  - {plot_file}" for plot_file in plot_files)
    
    # Save analysis results
    print("Saving analysis results...")
    study.save_analysis_results(params['output_dir'])
    
    # Generate hypothesis report
    print("Generating hypothesis evaluation report...")
    report = study.generate_sequential_report()
    
    # Save report
    report_path = f"{params['output_dir']}/sequential_convergence_hypothesis_report.txt"
    with open(report_path, 'w') as f:
        f.write(report)
    
    print(f"Hypothesis evaluation report saved to: {report_path}")
    
    # Display key findings
    print("\n" + "=" * 80)
    print("KEY FINDINGS SUMMARY")
    print("=" * 80)
    
    if 'hypothesis_support' in analysis_results:
        hyp_support = analysis_results['hypothesis_support']
        overall_support = hyp_support.get('overall_support', 'unknown')
        print(f"Overall Hypothesis Support: {overall_support.upper()}")
        
        evidence = hyp_support.get('evidence_strength', {})
        if 'sequential_index' in evidence:
            seq_info = evidence['sequential_index']
            print(f"Sequential Index: {seq_info['value']:.3f} ({seq_info['strength']})")
        
        if 'degeneracy_achievement' in evidence:
            deg_info = evidence['degeneracy_achievement']
            print(f"Degeneracy Achievement: {deg_info['value']:.1%} ({deg_info['strength']})")
        
        if 'temporal_separation' in evidence:
            temp_info = evidence['temporal_separation']
            print(f"Temporal Separation: {temp_info['value']:.3f} ({temp_info['strength']})")
    
    return analysis_results, plot_files

# Generate analysis and plots
analysis_results, plot_files = generate_analysis_and_plots(study, params)

## 5. Displaying Generated Plots

Let's display the key plots inline to see the results.

In [ ]:
# Display key plots inline
plots_dir = f"{params['output_dir']}/plots"

# List of key plots to display
key_plots = [
    'statistical_analysis.png',
    'convergence_timeline_analysis.png',
    'probability_evolution_analysis.png',
    'system_dynamics_analysis.png',
    'barycentric_all_trajectories.png'
]

print("Displaying key plots:")
for plot_name in key_plots:
    plot_path = f"{plots_dir}/{plot_name}"
    if os.path.exists(plot_path):
        print(f"\n{plot_name}:")
        img = plt.imread(plot_path)
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    else:
        print(f"\n{plot_name} not found at {plot_path}")

## 6. Output Structure

Here's what was generated and where to find it:

All results are saved in: **`<output_path>`**

## File structure: 
<output_path>/ 
├── plots/ \
│ ├── statistical_analysis.png \
│ ├── convergence_timeline_analysis.png \
│ ├── probability_evolution_analysis.png \
│ ├── system_dynamics_analysis.png \
│ ├── barycentric_all_trajectories.png \
│ ├── barycentric_individual_trajectories.png \
│ └── barycentric_final_distributions.png \
├── sequential_convergence_raw_data.csv \
├── analysis_results.json \
└── sequential_convergence_hypothesis_report.txt


## File descriptions:

- **`plots/`**: All generated visualisations  
- **`sequential_convergence_raw_data.csv`**: Raw experimental data  
- **`analysis_results.json`**: Statistical analysis results  
- **`sequential_convergence_hypothesis_report.txt`**: Detailed hypothesis evaluation

## Key plots explained:

- **`statistical_analysis.png`**: Comprehensive sequential convergence analysis  
- **`convergence_timeline_analysis.png`**: Temporal evolution of agent specialisation  
- **`probability_evolution_analysis.png`**: Learning dynamics and preference formation  
- **`system_dynamics_analysis.png`**: Environmental feedback and coordination  
- **`barycentric_all_trajectories.png`**: Agent trajectories in probability space


## 7. Tutorial Summary

This tutorial has successfully demonstrated the Sequential Convergence Study experiment. Here's what we accomplished:

### Key Findings:
- **Sequential Pattern**: Agents converge at distinct time points rather than simultaneously
- **Temporal Separation**: Perfect sequential indices (1.000) across all replications
- **Degeneracy Achievement**: Universal (100%) achievement of specialised probability distributions
- **Environmental Feedback**: Early convergers influence subsequent agent behaviour
- **Consistent Patterns**: Sequential convergence is a robust and predictable property

### What You Can Do Next:
- Examine the generated plots in the plots/ directory
- Review the hypothesis evaluation report for detailed findings
- Modify parameters and run again to explore different scenarios
- Use the raw data for further custom analysis
- Compare results with other experiments to understand system dynamics

### Expected Results:
The experiment typically shows:
1. **Sequential Index**: Perfect sequential separation (1.000) across all replications
2. **Degeneracy Achievement**: 100% of agents achieve specialised distributions
3. **Temporal Patterns**: Predictable convergence timing and ordering
4. **Environmental Feedback**: Early convergers create cost gradients for later agents
5. **Consistency**: Robust sequential patterns across diverse experimental conditions

All results have been saved to the specified output directory for further analysis.